## RQ1 Forest Definition Comparisons 

[Add description]

For analysis and comaprison steps - make visualisations!

Steps
1. Analyse & Compare FNF maps (6 maps total from steps 1 & 2) 
    1. For Germany
    2. For Natura 2000 areas (use zonal statistics)
2. Create *Forest Presence/Absence Consensus Map* (Sexton et al 2016 approach)
3. Analyse Consensus Map
    1. For Germany
    2. For Natura 2000 areas (use zonal statistics)


In [4]:
# SETUP

# Note: this .ipynb file depends on files & folder structures created in rq1_step1_data_prep.ipynb 
# and rq1_step2_fao_forest.ipynb

# Import packages
import subprocess
import pickle
import geopandas as gpd
import pandas as pd

from rasterstats import zonal_stats

# Store gdal.py paths
gdal_calc = "./thesis_env_conda/Lib/site-packages/GDAL-3.10.1-py3.12-win-amd64.egg-info/scripts/gdal_calc.py"

### Step 1: Analyse & Compare FNF Maps

**Natura 2000 Stats**
1. Zonal stats (sum and count) per geometry for each FNF map
2. Use the total forest (sum) and total area (count) values to calculate percentage forest for each geometry
3. Save above outputs (for loading later)

4. Sum the per geometry sum and count values to get the total forest and total area for all Natura 2000 areas, for each FNF map
5. Use the total forest and total area to get the percentage forest for each FNF map

5. Calculate the average forest area (all Natura 2000 areas) across all FNF maps (one result)
6. Calculate the difference from the average for each FNF map (step 4 total forest outputs minus step 5 output) 

**Germany Stats**
1. Calculate the total fores (sum) and total area (count) for whole raster for each FNF map
2. Save above outputs (for loading later)

3. Use total forest (sum) and total area (count) to calculate the percentage of forest for each FNF map.

4. Calculate the average forest area for Germany across all FNF maps (one result)
5. Calculate the difference from the average for each FNF map (step 1 total forest outputs minus step 4 output) 



Create visaulisations as well (maps and graphs)



In [ ]:
# 1: FNF NATURA ZONAL STATISTICS
# TAKES ABOUT 175 MIN

# Store paths to the FNF maps
esa_FNF = "./outputs/esa_lccs_class_3035_DE_5m_FNF.tif"
jaxa_FNF = "./outputs/jaxa_FNF_3035_DE_5m_FNF.tif"
corine_FNF = "./outputs/U2018_CLC2018_V2020_3035_DE_5m_FNF.tif"
ger_lulc_FNF = "./outputs/clc5_class3xx_3035_DE_5m_FNF.tif"
hansen_FNF = "./outputs/hansen_60cover_2018_3035_DE_5m_FNF.tif"
#fao_FNF = "./outputs/"

# Store the path to the German Natura 2000 areas
natura = gpd.read_file("./outputs/natura2000_3035_DE.shp")

# Calculate the sum and count for each Natura 2000 geometry, for each FNF map 
esaFNF_natura_stats = zonal_stats(natura, esa_FNF,
                                  stats=['sum', 'count'],
                                  geojson_out=True)
jaxaFNF_natura_stats = zonal_stats(natura, jaxa_FNF,
                                  stats=['sum', 'count'],
                                  geojson_out=True)
corineFNF_natura_stats = zonal_stats(natura, corine_FNF,
                                  stats=['sum', 'count'],
                                  geojson_out=True)
gerlulcFNF_natura_stats = zonal_stats(natura, ger_lulc_FNF,
                                  stats=['sum', 'count'],
                                  geojson_out=True)
hansenFNF_natura_stats = zonal_stats(natura, hansen_FNF,
                                  stats=['sum', 'count'],
                                  geojson_out=True)
#faoFNF_natura_stats = zonal_stats(natura, fao_FNF,
#                                  stats=['sum', 'count'],
#                                  geojson_out=True)



In [ ]:
# Store the stats as a dataframe
esaFNF_natura_stats = pd.DataFrame(esaFNF_natura_stats)
jaxaFNF_natura_stats = pd.DataFrame(jaxaFNF_natura_stats)
corineFNF_natura_stats = pd.DataFrame(corineFNF_natura_stats)
gerlulcFNF_natura_stats = pd.DataFrame(gerlulcFNF_natura_stats)
hansenFNF_natura_stats = pd.DataFrame(hansenFNF_natura_stats)
#faoFNF_natura_stats = pd.DataFrame(faoFNF_natura_stats)



In [ ]:
print(corineFNF_natura_stats[1:10])

In [11]:
# Save the zonal stats outputs 
pickle.dump(esaFNF_natura_stats, open("./outputs/esaFNF_natura_stats.p", "wb"))
pickle.dump(jaxaFNF_natura_stats, open("./outputs/jaxaFNF_natura_stats.p", "wb"))
pickle.dump(corineFNF_natura_stats, open("./outputs/corineFNF_natura_stats.p", "wb"))
pickle.dump(gerlulcFNF_natura_stats, open("./outputs/gerlulcFNF_natura_stats.p", "wb"))
pickle.dump(hansenFNF_natura_stats, open("./outputs/hansenFNF_natura_stats.p", "wb"))
#pickle.dump(faoFNF_natura_stats, open("./outputs/faoFNF_natura_stats.p", "wb"))

In [12]:
# Load the zonal stats outputs 
esaFNF_natura_stats = pickle.load(open("./outputs/esaFNF_natura_stats.p", "rb"))
jaxaFNF_natura_stats = pickle.load(open("./outputs/jaxaFNF_natura_stats.p", "rb"))
corineFNF_natura_stats = pickle.load(open("./outputs/corineFNF_natura_stats.p", "rb"))
gerlulcFNF_natura_stats = pickle.load(open("./outputs/gerlulcFNF_natura_stats.p", "rb"))
hansenFNF_natura_stats = pickle.load(open("./outputs/hansenFNF_natura_stats.p", "rb"))
#faoFNF_natura_stats = pickle.load(open("./outputs/faoFNF_natura_stats.p", "rb"))

In [ ]:
print(corineFNF_natura_stats[1:10])

### Step 2: Create Forest Presence/Absence Consensus map 

Sexton et al 2016 approach - basically just sum the FNF maps

In [ ]:
# 2: FOREST CONSENSUS MAP

# Store paths to the FNF maps
esa_FNF = "./outputs/esa_lccs_class_3035_DE_5m_FNF.tif"
jaxa_FNF = "./outputs/jaxa_FNF_3035_DE_5m_FNF.tif"
corine_FNF = "./outputs/U2018_CLC2018_V2020_3035_DE_5m_FNF.tif"
ger_lulc_FNF = "./outputs/clc5_class3xx_3035_DE_5m_FNF.tif"
hansen_FNF = "./outputs/hansen_60cover_2018_3035_DE_5m_FNF.tif"
#fao_FNF = "./outputs/"

# Runs gdal_calc.py to add the input rasters together (ABOUT 15 MIN w 5 rasters)
consensus_map = subprocess.run(['python', 
                                gdal_calc, 
                                '-A', esa_FNF, 
                                '-B', jaxa_FNF, 
                                '-C', corine_FNF, 
                                '-D', ger_lulc_FNF, 
                                '-E', hansen_FNF,
                                '--outfile=./outputs/test_consensus_map_v2.tif', 
                                '--calc=A+B+C+D+E', 
                                '--co=COMPRESS=LZW', 
                                '--co=BIGTIFF=YES', 
                                '--NoDataValue=-9999'
                                ],
                                capture_output=True, 
                                text=True)

print(consensus_map.stdout)
print(consensus_map.stderr)


### Step 3: Analyse Consensus Map

See notes and ideas for visualisations in "writing" folder

Create visaulisations as well (maps and graphs)